In [ ]:
import torch

In [ ]:
# prompt = """<|im_start|>system
# You are a helpful assistant.</s>
# <|im_start|>user
# Hello world</s>
# <|im_start|>assistant
# Hi there, how can I help?</s>

# # ! ENSURE 1 and only 1 bos `<s>` at the beginning of sequence
# print(tokenizer.convert_ids_to_tokens(tokenizer.encode(prompt)))

# ['<s>', '▁<', '|', 'im', '_', 'start', '|', '>', 'system', '<0x0A>', 'You', '▁are', '▁a', '▁helpful', '▁assistant', '.', '</s>', '▁', '<0x0A>', '<', '|', 'im', '_', 'start', '|', '>', 'user', '<0x0A>', 'Hello', '▁world', '</s>', '▁', '<0x0A>', '<', '|', 'im', '_', 'start', '|', '>', 'ass', 'istant', '<0x0A>', 'Hi', '▁there', ',', '▁how', '▁can', '▁I', '▁help', '?', '</s>', '▁', '<0x0A>']
# """


In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # device = "cuda" # the device to load the model onto
# device = "cpu"

# # model = AutoModelForCausalLM.from_pretrained("SeaLLMs/SeaLLM-7B-v2", torch_dtype=torch.bfloat16, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2")

# model = AutoModelForCausalLM.from_pretrained(
#     "LoneStriker/SeaLLM-7B-v2-GGUF", 
#     model_file="SeaLLM-7B-v2-Q8_0.gguf",
#     device_map=device
# )
# # tokenizer = AutoTokenizer.from_pretrained("LoneStriker/SeaLLM-7B-v2-GGUF")

# messages = [
#     {"role": "user", "content": "Hello world"},
#     {"role": "assistant", "content": "Hi there, how can I help you today?"},
#     {"role": "user", "content": "Explain general relativity in details."},
# ]

# encodeds = tokenizer.apply_chat_template(
#     messages, return_tensors="pt", add_generation_prompt=True
# )
# print(tokenizer.convert_ids_to_tokens(encodeds[0]))
# # ['<s>', '▁<', '|', 'im', '_', 'start', '|', '>', 'user', '<0x0A>', 'Hello', '▁world', '</s>', '▁', '<0x0A>', '<', '|', 'im ....

# model_inputs = encodeds.to(device)
# model.to(device)

# generated_ids = model.generate(
#     model_inputs,
#     max_new_tokens=1000,
#     do_sample=True,
#     pad_token_id=tokenizer.pad_token_id,
# )
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])

In [ ]:
## Imports
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Download the GGUF model
# model_name = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF"
# model_file = "mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf" # this is the specific model file we'll use in this example. It's a 4-bit quant, but other levels of quantization are available in the model repo if preferred
model_name = "LoneStriker/SeaLLM-7B-v2-GGUF"
model_file = "SeaLLM-7B-v2-Q8_0.gguf"
model_path = hf_hub_download(model_name, filename=model_file)

## Instantiate model from downloaded file
llm = Llama(
    model_path=model_path,
    n_ctx=16000,  # Context length to use
    n_threads=32,            # Number of CPU threads to use
    n_gpu_layers=0        # Number of model layers to offload to GPU
)

## Generation kwargs
generation_kwargs = {
    "max_tokens":2000,
    "stop":["</s>"],
    "echo":False, # Echo the prompt in the output
    "top_k":1 # This is essentially greedy decoding, since the model will always return the highest-probability token. Set this value > 1 for sampling decoding
}

In [ ]:
## Run inference
# prompt = "The meaning of life is "
prompt = "1+1=?"
res = llm(prompt, **generation_kwargs) # Res is a dictionary

## Unpack and the generated text from the LLM response dictionary and print it
print(res["choices"][0]["text"])